In [ ]:
pip install -Uqq fastai

     |████████████████████████████████| 188 kB 31.0 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 


In [ ]:
import os
import json
import gzip
import torch
import pandas as pd
import numpy as np

from urllib.request import urlopen
from tqdm.autonotebook import trange
from fastai.text.all import *

# Get and parse data

In [ ]:
# Must have filled a form to use the data: https://forms.gle/A8hBfPxKkKGFCP238

!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Books.json.gz

--2021-07-25 10:11:58--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Books.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1219104464 (1.1G) [application/octet-stream]
Saving to: ‘meta_Books.json.gz.1’

meta_Books.json.gz. 100%[===================>]   1.13G  39.9MB/s    in 30s     

2021-07-25 10:12:27 (39.3 MB/s) - ‘meta_Books.json.gz.1’ saved [1219104464/1219104464]



In [ ]:
# Procedure to get metadata for one book.

with gzip.open('/content/meta_Books.json.gz') as g:
  for l in g:
    print (json.loads(l))
    break

{'category': [], 'tech1': '', 'description': ['It is a biology book with God&apos;s perspective.'], 'fit': '', 'title': 'Biology Gods Living Creation Third Edition 10 (A Beka Book Science Series)', 'also_buy': ['0669009075', 'B000K2P5SA', 'B00MD4G2N0', 'B000ASIPTK', '0130508470', '1892427524', '0321567919', 'B000BJBH20', '0547484631', 'B000HAJTQO', 'B000AUCX7I', '0130365645', 'B000BI1Y2O', '0395976715', '052817729X', '1579246443', 'B001CK63XK', '1591669847', '0395879884', '836585161X', 'B01J2F9BH6', 'B00KYEHR4E', '158008141X', '1857928393', '0927545829', 'B015AR0RA0', 'B000TVHHRE', '0865167990', '1579246052', 'B003NXXVD4', 'B000OH6AX0', '061802087X', 'B000NU2X02', '0743252012'], 'tech2': '', 'brand': 'Keith Graham', 'feature': [], 'rank': '1,349,781 in Books (', 'also_view': ['0019777701', 'B000AUCX7I', 'B000K2P5SA', 'B001CK63XK', 'B01J2F9BH6', 'B000BI1Y2O', '1932012540', 'B0095ZCRCK'], 'main_cat': 'Books', 'similar_item': '', 'date': '', 'price': '$39.94', 'asin': '0000092878', 'image

In [ ]:
def parse(path):
  """
  Open the JSON file and yield every line.
  """
  with gzip.open(path) as g:
    for l in g:
      yield json.loads(l)

def getDF(path, max_items=None):
  """
  Get `max_items` and turn them into a pandas DataFrame. 
  """
  i = 0
  df = {}
  for d in parse(path):
    if max_items and i == max_items:
      break
    df[i] = {}    
    try:
      df[i]['title'] = d['title']
    except KeyError:
      pass
    try:
      df[i]['description'] = d['description'][0] 
    except IndexError: 
      pass
    i += 1
  return pd.DataFrame.from_dict(df, orient='index', columns=['title', 'description'])

In [ ]:
df = getDF('/content/meta_Books.json.gz', max_items=None)

In [ ]:
df

,title,description
0,Biology Gods Living Creation Third Edition 10 (A Beka Book Science Series),It is a biology book with God&apos;s perspective.
1,"Flex! Discography of North American Punk, Hardcore, and Powerpop 1975-1985 A-M","Discography of American Punk, Hardcore, and Power Pop"
2,Heavenly Highway Hymns: Shaped-Note Hymnal,This is a collection of classic gospel hymns that many churches still enjoy singing today.
3,"Principles of Analgesic Use in the Treatment of Acute Pain and Cancer Pain (APS, Principles of Analgesic Use in the Treatment of Acute Pain and Cancer Pain)",Brand new; never used.
4,MKSAP 15 Audio Companion,Flash cards used with accompany MKSAP 15 audio companion. \nExtremely useful for board exam.
...,...,...
2384192,The Walking Dead The Official Magazine Issue # 17 Newsstand Cover,The OFFICIAL THE WALKING DEAD MAGAZINE #17. 100 PAGE ISSUE!!! Feature Interviews with Cast and Crew plus TONS MORE Loaded with Full Color Photos front to back.Comes with a special insert called The Walking Dead Collector's Models.
2384193,Busoni : Konzertstuck fur Klavier mit Orchester,"Busoni, Ferruccio : Konzertstck fr Klavier mit Orchester. Op. 31a. Fr zwei Klaviere zu vier Hnden. (2. Klavier an Stelle des Orchesters). Piano Music - 2-Piano Scores This is an Eastman Scores Publishing professional reprint of the work originally published by: Breitkopf & Hartel, Leipzig, 1892, 2 scores, 38 pp. Sheet Music Eastman Scores Publishing Library Commerce ISMN : 979-0-087-00129-8"
2384194,Entertainment Weekly Magazine July 1 2016 | Star Wars Rogue One,New Entertainment Weekly Magazine July 1 2016 | Star Wars Rogue One
2384195,PLAYBOY MAGAZINE JULY/AUGUST 2016,"this product was never view, only taken out of plastic cover."


In [ ]:
print(df.isna().sum())
df[df['description'].isna()]

title               0
description    550752
dtype: int64


,title,description
1,Mksap 16 Audio Companion: Medical Knowledge Self-Assessment Program,NaN
4,Georgina Goodman Nelson Womens Size 8.5 Purple Regular Suede Platforms Shoes,NaN
9,LJ Classique Interchangeable Ladies Gift Set Watches,NaN
10,Classic Soul Winner's New Testament Bible,NaN
13,"The New England Historical and Genealogical Register, Vol 155, April 2001 (618)",NaN
...,...,...
2934944,Made Men: A Thriller (Law of Retaliation Book 2) - Kindle edition,NaN
2934945,Raptor&#39;s Desire (A Planet Desire novelette) - Kindle edition,NaN
2934946,"LG K4 Case,LG Optimus Zone 3 Case,LG Spree Case,LG Rebel LTE Case,Veggzy[Kickstand]Slim Heavy Duty Shock Absorption Rugged Armor Dual Layer Hybird High Impact Resistant Defender Protective Hard Cover",NaN
2934947,Magickal Incantations,NaN


In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,title,description
0,Biology Gods Living Creation Third Edition 10 (A Beka Book Science Series),It is a biology book with God&apos;s perspective.
1,"Flex! Discography of North American Punk, Hardcore, and Powerpop 1975-1985 A-M","Discography of American Punk, Hardcore, and Power Pop"
2,Heavenly Highway Hymns: Shaped-Note Hymnal,This is a collection of classic gospel hymns that many churches still enjoy singing today.
3,"Principles of Analgesic Use in the Treatment of Acute Pain and Cancer Pain (APS, Principles of Analgesic Use in the Treatment of Acute Pain and Cancer Pain)",Brand new; never used.
4,MKSAP 15 Audio Companion,Flash cards used with accompany MKSAP 15 audio companion. \nExtremely useful for board exam.
...,...,...
2384192,The Walking Dead The Official Magazine Issue # 17 Newsstand Cover,The OFFICIAL THE WALKING DEAD MAGAZINE #17. 100 PAGE ISSUE!!! Feature Interviews with Cast and Crew plus TONS MORE Loaded with Full Color Photos front to back.Comes with a special insert called The Walking Dead Collector's Models.
2384193,Busoni : Konzertstuck fur Klavier mit Orchester,"Busoni, Ferruccio : Konzertstck fr Klavier mit Orchester. Op. 31a. Fr zwei Klaviere zu vier Hnden. (2. Klavier an Stelle des Orchesters). Piano Music - 2-Piano Scores This is an Eastman Scores Publishing professional reprint of the work originally published by: Breitkopf & Hartel, Leipzig, 1892, 2 scores, 38 pp. Sheet Music Eastman Scores Publishing Library Commerce ISMN : 979-0-087-00129-8"
2384194,Entertainment Weekly Magazine July 1 2016 | Star Wars Rogue One,New Entertainment Weekly Magazine July 1 2016 | Star Wars Rogue One
2384195,PLAYBOY MAGAZINE JULY/AUGUST 2016,"this product was never view, only taken out of plastic cover."


# Fine-tune Language Model

## Create DataLoaders and Learner

In [ ]:
rand_num_gen = np.random.default_rng(42)
random_indices = rand_num_gen.integers(0, len(df), size=400_000)
sampled_df = df.iloc[list(set(random_indices))]

In [ ]:
dls = TextDataLoaders.from_df(sampled_df, text_col=1, is_lm=True)
dls.show_batch(max_n=3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,"xxbos < span><span > xxunk 's poignant book delivers the fullest account yet of xxmaj bobby xxmaj fischer 's triumphal and tragic years in xxmaj iceland with a thoughtful , thorough , and balanced presentation , including topics that transport us beyond the limits of its place and theme . a personal and heartbreaking account . \n < / span > < / span > ( john xxup d. xxmaj xxunk ,","< span><span > xxunk 's poignant book delivers the fullest account yet of xxmaj bobby xxmaj fischer 's triumphal and tragic years in xxmaj iceland with a thoughtful , thorough , and balanced presentation , including topics that transport us beyond the limits of its place and theme . a personal and heartbreaking account . \n < / span > < / span > ( john xxup d. xxmaj xxunk , xxmaj"
1,"joukhadar ’s intoxicating debut , the past and present are brought to life , illuminating how , in exile , neither can exist without the other . xxmaj with clear , exquisite prose , xxmaj joukhadar unspools a brightly imagined tale of family and grief , mapmaking and migration . xxmaj this important book is a love letter to the vanished — and to what remains . ” ( xxunk xxmaj xxunk","’s intoxicating debut , the past and present are brought to life , illuminating how , in exile , neither can exist without the other . xxmaj with clear , exquisite prose , xxmaj joukhadar unspools a brightly imagined tale of family and grief , mapmaking and migration . xxmaj this important book is a love letter to the vanished — and to what remains . ” ( xxunk xxmaj xxunk ,"
2,"as it depicts the plants and animals simply referred to as "" someone "" or "" something . "" xxmaj the humorous creatures busily interpret the words and images in an entertaining manner . xxmaj useful for early poetry study or read - aloud xxunk mcgrath , xxmaj clearwater xxmaj public xxmaj library , xxup fl \n xxmaj copyright 1997 xxmaj reed xxmaj business xxmaj information , xxmaj inc . xxbos xxup","it depicts the plants and animals simply referred to as "" someone "" or "" something . "" xxmaj the humorous creatures busily interpret the words and images in an entertaining manner . xxmaj useful for early poetry study or read - aloud xxunk mcgrath , xxmaj clearwater xxmaj public xxmaj library , xxup fl \n xxmaj copyright 1997 xxmaj reed xxmaj business xxmaj information , xxmaj inc . xxbos xxup about"


In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]).to_fp16()

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:120: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 2e-3)

## Test tokenizer and numericalization

In [ ]:
row = df.iloc[0]['description']

In [ ]:
a = dls.tokenizer(row3)
a_n = dls.numericalize(a)

# Get embeddings for all the book descriptions 

In [ ]:
# Super slow
# learn.model.eval()
# embds = np.empty((len(df), 400))
# i = 0

# for description in df['description']:
#   learn.model.reset()
#   num_description = get_tokenized_description(dls, description)
#   embeddings = learn.model[0](num_description[None])[0]
#   mean_embeddings = embeddings.mean(0)
#   embds[i,:] = mean_embeddings.cpu().data.numpy()
#   i += 1
#   print(i)

In [ ]:
def get_numericalized_description(dls, sentences):
  num_sentences = []
  lengths = []
  for sentence in sentences:
    tok_description = dls.tokenizer(sentence)
    num_description = dls.numericalize(tok_description)
    num_sentences.append(num_description)
    lengths.append(len(num_description))
  return num_sentences, max(lengths)

In [ ]:
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for element in batch:
        if isinstance(element, Tensor):
            element = element.to(target_device)
    return batch

In [ ]:
def pad_to_max_length(batch, max_length):
  tensors = []
  for tensor in batch:
    tensor = torch.nn.functional.pad(tensor, (0, max_length-len(tensor)), value=1)
    tensors.append(tensor)
  return tensors

In [ ]:
learn.model.eval()
embds = np.empty((len(df), 400))

def get_numpy_embeddings(df, model, dls, batch_size=32, device='cuda'):
  for start_index in trange(0, len(df), batch_size):
    if (start_index + batch_size) < len(df):
      sentences_batch = df['description'][start_index:start_index+batch_size]
    else:
      sentences_batch = df['description'][start_index:]
    features, max_length = get_numericalized_description(dls, sentences_batch)
    features = batch_to_device(features, device)
    features = pad_to_max_length(features, max_length)
    features = torch.stack(features)
    model.reset()
    embeddings = model[0](features)
    mean_embeddings = embeddings.mean(1)
    if (start_index + batch_size) < len(df):
      embds[start_index:start_index+batch_size,:] = mean_embeddings.cpu().data.numpy()
    else:
      sentences_batch = df['description'][start_index:]
      embds[start_index:,:] = mean_embeddings.cpu().data.numpy()

In [ ]:
learn.model.eval()

def get_numpy_embeddings(df, model, dls, batch_size=32, device='cuda'):
    embds = np.empty((len(df), 400))
    for start_index in trange(0, len(df), batch_size):
        if (start_index + batch_size) < len(df):
            sentences_batch = df['description'][start_index:start_index+batch_size]
        else:
            sentences_batch = df['description'][start_index:]
        features, max_length = get_numericalized_description(dls, sentences_batch)
        features = batch_to_device(features, device)
        features = pad_to_max_length(features, max_length)
        features = torch.stack(features)
        model.reset()
        embeddings = model[0](features)
        mean_embeddings = embeddings.mean(1)
        if (start_index + batch_size) < len(df):
            embds[start_index:start_index+batch_size,:] = mean_embeddings.cpu().data.numpy()
        else:
            embds[start_index:,:] = mean_embeddings.cpu().data.numpy()
    return embds

In [ ]:
embds = get_numpy_embeddings(df[:2_600_000], learn.model, dls, batch_size=50, device='cpu')

In [ ]:
learn.model[0](a_n[None]).shape

torch.Size([1, 250, 400])

# Save components for the application

In [ ]:
torch.save(learn, 'fastai_learner.pt')

Path('models/test_model.pth')

In [ ]:
torch.save(learn.model[0], 'fastai_model.pt')

In [ ]:
torch.save(dls.tokenizer, 'fastai_tokenizer.pt')

In [ ]:
torch.save(dls.numericalize, 'fastai_numericalize.pt')

In [ ]:
np.save('fastai_embeddings', embds)

# Test API endpoint

In [ ]:
import requests
import json
import pandas as pd
input_json = json.dumps({"search_string": "learn numbers", "ID": "111"})
# Call the endpoint
endpoint = 'http://50a9efd4-bbc1-4e53-b376-f44aac367846.southcentralus.azurecontainer.io/score'
headers = { 'Content-Type':'application/json'} #prod wrap
resp = requests.post(endpoint, input_json, headers=headers)
print(resp)
results = resp.text
print(results)
data = json.loads(results)
df = pd.read_json(data['response'])
print(df)

<Response [200]>
{"search_string": "learn numbers", "ID": "111", "response": "{\"title\":{\"2606545\":\"Theory of Numbers.\",\"2611470\":\"Theory of Numbers.\",\"8259\":\"Virginia Math Connects Homework and Problem-Solving Practice Workshop Grade 3\",\"280055\":\"Mathematics: The Path to Math Success! (Teacher Guide) Grade 5, Volume 2\",\"110499\":\"Harcourt Horizons Texas &amp; Student Edition, Grade 4 (Harcourt School Publishers Horizons)\",\"640839\":\"Steck-Vaughn Working With Numbers: Algebra\",\"611577\":\"Mental Arithmetic 1 (Bk. 1)\",\"725808\":\"Nimble With Numbers, Grades 5 &amp; 6\",\"713877\":\"Excel Charts For Dummies\",\"2616777\":\"Dialogues on mathematics\"},\"description\":{\"2606545\":\"Theory of Numbers.\",\"2611470\":\"Theory of Numbers.\",\"8259\":\"Math\",\"280055\":\"math\",\"110499\":\"Math\",\"640839\":\"ALGEBRA WORKING WITH NUMBERS\",\"611577\":\"Mental Arithmetic Book 1\",\"725808\":\"numbers\",\"713877\":\"Develop charts that help you explain the numbers\",\